<a href="https://colab.research.google.com/github/RohanOpenSource/Deep-Learning-And-Beyond/blob/main/HarryPotterLSTM%3C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ON THIS FINE AND BEAUTIFUL DAY, we'll be making an lstm that generates text in the style of a harry potter script! I found this dataset yesterday and couldn't stop thinking about it haha.
Here's the link: https://www.kaggle.com/datasets/gulsahdemiryurek/harry-potter-dataset?select=Harry+Potter+1.csv
I will also try to use it to make a realistic sorting hat, but that may prove to be a bit complicated.

In [ ]:
text = open("hp1.txt").read()
n_chars = len(text)
vocab = ''.join(sorted(set(text)))
print("unique_chars:", vocab)
n_unique_chars = len(vocab)
print("Number of characters:", n_chars)
print("Number of unique characters:", n_unique_chars)

unique_chars: 	
 !"&',-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz} –—’“”…﻿
Number of characters: 202668
Number of unique characters: 85


In [ ]:
char2int = {c: i for i, c in enumerate(vocab)}
int2char = {i: c for i, c in enumerate(vocab)}

In [ ]:
import pickle
import os
BASENAME = os.path.basename(os.path.basename("hp1.txt"))
pickle.dump(char2int, open(f"{BASENAME}-char2int.pickle", "wb"))
pickle.dump(int2char, open(f"{BASENAME}-int2char.pickle", "wb"))

In [ ]:
import numpy as np
import tensorflow as tf
encoded_text = np.array([char2int[c] for c in text])
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
for char in char_dataset.take(10):
    print(char.numpy(), int2char[char.numpy()])

84 ﻿
26 C
57 h
50 a
67 r
50 a
52 c
69 t
54 e
67 r


In [ ]:
sequences = char_dataset.batch(2*200 + 1, drop_remainder=True)
for sequence in sequences.take(2):
    print(''.join([int2char[i] for i in sequence.numpy()]))

﻿Character;Sentence
Dumbledore;I should've known that you would be here, Professor McGonagall.
McGonagall;Good evening, Professor Dumbledore.
McGonagall;Are the rumors true, Albus?
Dumbledore;I'm afraid so, professor.
Dumbledore;The good and the bad.
McGonagall;And the boy?
Dumbledore;Hagrid is bringing him.
McGonagall;Do you think it wise to trust Hagrid with something as important as this?
Dumble
dore;Ah, Professor, I would trust Hagrid with my life.
Hagrid;Professor Dumbledore, sir.
Hagrid;Professor McGonagall.
Dumbledore;No problems, I trust, Hagrid?
Hagrid;No, sir.
Hagrid;Little tyke fell asleep just as we were flying over Bristol.
Hagrid;Try not to wake him.
Hagrid;There you go.
Dumbledore;Albus, do you really think it's safe, leaving him with these people?
McGonagall;I've watched them 


In [ ]:
def split_sample(sample):
    ds = tf.data.Dataset.from_tensors((sample[:200], sample[200]))
    for i in range(1, (len(sample)-1) // 2):
        input_ = sample[i: i+200]
        target = sample[i+200]
        other_ds = tf.data.Dataset.from_tensors((input_, target))
        ds = ds.concatenate(other_ds)
    return ds

dataset = sequences.flat_map(split_sample)

In [ ]:
def one_hot_samples(input_, target):
    return tf.one_hot(input_, n_unique_chars), tf.one_hot(target, n_unique_chars)

dataset = dataset.map(one_hot_samples)

In [ ]:
for element in dataset.take(2):
    print("Input:", ''.join([int2char[np.argmax(char_vector)] for char_vector in element[0].numpy()]))
    print("Target:", int2char[np.argmax(element[1].numpy())])
    print("Input shape:", element[0].shape)
    print("Target shape:", element[1].shape)
    print("="*50, "\n")

Input: ﻿Character;Sentence
Dumbledore;I should've known that you would be here, Professor McGonagall.
McGonagall;Good evening, Professor Dumbledore.
McGonagall;Are the rumors true, Albus?
Dumbledore;I'm afra
Target: i
Input shape: (200, 85)
Target shape: (85,)

Input: Character;Sentence
Dumbledore;I should've known that you would be here, Professor McGonagall.
McGonagall;Good evening, Professor Dumbledore.
McGonagall;Are the rumors true, Albus?
Dumbledore;I'm afrai
Target: d
Input shape: (200, 85)
Target shape: (85,)



In [ ]:
ds = dataset.repeat().shuffle(1024).batch(128, drop_remainder=True)

Time for the actual LSTM

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, input_shape=(200, n_unique_chars), return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(256),
    tf.keras.layers.Dense(n_unique_chars, activation="softmax"),
])
model_weights_path = f"results/{BASENAME}-{200}.h5"
model.summary()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 200, 256)          350208    
                                                                 
 dropout (Dropout)           (None, 200, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 85)                21845     
                                                                 
Total params: 897,365
Trainable params: 897,365
Non-trainable params: 0
_________________________________________________________________


In [16]:
if not os.path.isdir("results"):
    os.mkdir("results")
model.fit(ds, steps_per_epoch=(len(encoded_text) - 200) // 128, epochs=30)
model.save(model_weights_path)

Epoch 1/30
1581/1581 [==============================] - 121s 76ms/step - loss: 0.5129 - accuracy: 0.8293
Epoch 2/30
1581/1581 [==============================] - 122s 77ms/step - loss: 0.5044 - accuracy: 0.8318
Epoch 3/30
1581/1581 [==============================] - 118s 75ms/step - loss: 0.4924 - accuracy: 0.8366
Epoch 4/30
1581/1581 [==============================] - 118s 75ms/step - loss: 0.4821 - accuracy: 0.8393
Epoch 5/30
1581/1581 [==============================] - 119s 75ms/step - loss: 0.4717 - accuracy: 0.8423
Epoch 6/30
1581/1581 [==============================] - 119s 75ms/step - loss: 0.4646 - accuracy: 0.8455
Epoch 7/30
1581/1581 [==============================] - 119s 75ms/step - loss: 0.4546 - accuracy: 0.8480
Epoch 8/30
1581/1581 [==============================] - 118s 75ms/step - loss: 0.4487 - accuracy: 0.8497
Epoch 9/30
1581/1581 [==============================] - 118s 75ms/step - loss: 0.4405 - accuracy: 0.8520
Epoch 10/30
1581/1581 [==============================] 

In [28]:
import tqdm
seed = "Hermione;How can you be nearly headless?"
s = seed
n_chars = 400
# generate 400 characters
generated = ""
for i in tqdm.tqdm(range(n_chars), "Generating text"):
    # make the input sequence
    X = np.zeros((1, 200, n_unique_chars))
    for t, char in enumerate(seed):
        X[0, (200 - len(seed)) + t, char2int[char]] = 1
    # predict the next character
    predicted = model.predict(X, verbose=0)[0]
    # converting the vector to an integer
    next_index = np.argmax(predicted)
    # converting the integer to a character
    next_char = int2char[next_index]
    # add the character to results
    generated += next_char
    # shift seed and the predicted character
    seed = seed[1:] + next_char

print("Seed:", s)
print("Generated text:")
print(generated)

Generating text: 100%|██████████| 400/400 [00:21<00:00, 18.75it/s]

Seed: Hermione;How can you be nearly headless?
Generated text:

Hagrid;No, thanks, I'll stand at the request of the 3rrisher.
Percy;Oh, We's come on now, him!
Hagrid;I was my family for?
Hermione;All right, well, he's got to hear before?
Hagrid;Now, uh, you could be always fears the broom?
Hermione;Alohomora
Hermione;Get in
Ron;Allast prenewt parents fought against him.
Hagrid;But nobody lived once.
Hagrid;Hello, professor.
Percy;Oh, and keep and please that 


BEAUTIFUL! IT'S BETTER THAN CURSED CHILD! Ok maybe this isn't practical, but messing around with this is hilarious so mission accomplished.